## Implementação dos modelos de linguagem

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai.text.models import * 

[`text.models`](/text.models.html#text.models) módulo implementa totalmente o codificador para um [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf), o [transformer model](https://arxiv.org/abs/1706.03762) e o [transformer XL model](https://arxiv.org/abs/1901.02860). Eles podem então ligado com um descodificador para fazer um modelo de linguagem, ou algumas camadas de classificação para fazer um classificador de texto.

## módulos de modelo Idioma

In [ ]:
show_doc(AWD_LSTM, title_level=3)

<h3 id="AWD_LSTM" class="doc_header"><code>class</code> <code>AWD_LSTM</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L75" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AWD_LSTM-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AWD_LSTM</code>(**`vocab_sz`**:`int`, **`emb_sz`**:`int`, **`n_hid`**:`int`, **`n_layers`**:`int`, **`pad_token`**:`int`=***`1`***, **`hidden_p`**:`float`=***`0.2`***, **`input_p`**:`float`=***`0.6`***, **`embed_p`**:`float`=***`0.1`***, **`weight_p`**:`float`=***`0.5`***, **`qrnn`**:`bool`=***`False`***, **`bidir`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="AWD_LSTM-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AWD_LSTM-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AWD_LSTM</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182.  

A idéia principal do artigo é usar um [RNN](http://www.pnas.org/content/79/8/2554) ao abandono em todos os lugares, mas de uma forma inteligente. Há uma diferença com o abandono de costume, razão pela qual você verá um módulo [`RNNDropout`](/text.models.awd_lstm.html#RNNDropout): nós zerar as coisas, como é habitual em abandono, mas nós sempre zero a mesma coisa de acordo com a dimensão seqüência (que é a primeira dimensão em pytorch ). Isso garante consistência ao atualizar o estado oculto através das frases inteiras / artigos.
Esta sendo dada, há um total de quatro desistências diferentes no codificador da AWD-LSTM:
- a primeira, a incorporação de abandono, é aplicada quando olhamos os ids de nossas fichas no interior da matriz de encastre (para transformá-los a partir de números de um vector de flutuador). Nós zerar algumas linhas do mesmo, assim ids aleatórios são enviados para um vetor de zeros em vez de serem enviados para o seu vector de incorporação. Este é o parâmetro `embed_p`.
- o segundo, o abandono de entrada, é aplicada ao resultado da incorporação com abandono. Esquecemo-nos de pedaços aleatórios de matriz incorporação (mas como indicado no último parágrafo, os mesmos na dimensão seqüência). Este é o parâmetro `input_p`.
- o terceiro é o abandono peso. É o mais complicado de implementar como podemos substituir aleatoriamente por 0s alguns pesos da matriz oculta-se escondido dentro da RNN: isso precisa ser feito de uma forma que garantir os gradientes ainda são computados e os pesos iniciais ainda atualizado. Este é o parâmetro `weight_p`.
- o quarto é o abandono escondido. É aplicada à saída de uma das camadas da RNN antes de ser usado como entrada da camada próxima (novamente mesmas coordenadas são postas em zero na dimensão sequência). Não é aplicado à última saída (que vai ter o seu próprio abandono no decodificador) .Este é o parâmetro `hidden_p`.
Os outros atributos são `vocab_sz` para o número de fichas na sua vocabulário,` emb_sz` para o tamanho incorporação, `n_hid` para o tamanho escondido dos seus LSTMs interiores (ou QRNNs),` n_layers` o número de camadas e `pad_token `para o índice de um símbolo de enchimento eventual (1 por padrão em fastai).
A bandeira `qrnn = true` substituir os LSTMs interiores por [QRNNs](https://arxiv.org/abs/1611.01576).

In [ ]:
show_doc(AWD_LSTM.reset)

<h4 id="AWD_LSTM.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L130" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AWD_LSTM-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="AWD_LSTM-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AWD_LSTM-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reset the hidden states.  

In [ ]:
show_doc(Transformer, title_level=3)

<h3 id="Transformer" class="doc_header"><code>class</code> <code>Transformer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L149" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Transformer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>Transformer</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***, **`learned_pos_enc`**:`bool`=***`True`***, **`mask`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="Transformer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Transformer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>Transformer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Transformer model: https://arxiv.org/abs/1706.03762.  

A idéia principal deste artigo é a utilização de rede neural regular para NLP em vez de uma RNN, mas com muitas camadas de atenção. Intuitivamente, essas camadas de atenção dizer o modelo que pagar mais juros a este ou àquele mundo quando se tenta prever a sua saída.
Ele começa a partir de mergulhos de `vocab_sz` (número de fichas) para` d_model` (que é basicamente o tamanho escondido em todo o modelo), e ele vai olhar para entradas de tamanho batch_size por `ctx_len` (para o comprimento do contexto). Nós adicionamos uma codificação posicional para os mergulhos (uma vez que uma rede neural normal não tem idéia da ordem de palavras), seja aprendido ou provenientes de [`PositionalEncoding`](/text.models.transformer.html#PositionalEncoding) dependendo `learned_pos_enc`. Temos, então, uma saída de `embed_p` seguido por 'blocos n_layers` de [`MultiHeadAttention`](/text.models.transformer.html#MultiHeadAttention) seguido por [`feed_forward`](/text.models.transformer.html#feed_forward).
Na atenção que usar `n_heads` com cada um estado oculto de` d_head` (será o padrão para `d_model // n_heads`). Se `máscara = true`, uma máscara irá certificar-se nenhuma atenção é dada para os tokens futuros (que seria batota ao treinar um modelo de linguagem). Se `escala = TRUE, as pontuações de atenção são escalonadas por um factor de 1` / Math.sqrt (d_head) `. Um abandono do `attn_p` é aplicada às contagens de atenção, então o resultado final se aplicada uma saída de` resid_p` antes de serem somadas para a entrada original (ligação residual antes de a norma camada).
Na alimentação para a frente, que tem duas camadas lineares de d_model` `a` d_inner` e, em seguida, de volta. Aqueles têm `bias` se que a bandeira é` true` e uma saída de `ff_p` é aplicada, depois de cada um, se double_drop` = TRUE, ou apenas no final de outra forma. `Act` é utilizada no meio como um não-linearidade.

In [ ]:
show_doc(TransformerXL, title_level=3)

<h3 id="TransformerXL" class="doc_header"><code>class</code> <code>TransformerXL</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L175" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#TransformerXL-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>TransformerXL</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`False`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadRelativeAttention'`***, **`learned_pos_enc`**:`bool`=***`False`***, **`mask`**:`bool`=***`True`***, **`mem_len`**:`int`=***`0`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="TransformerXL-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#TransformerXL-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>TransformerXL</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

TransformerXL model: https://arxiv.org/abs/1901.02860.  

TransformerXL é uma arquitetura de transformador com um tipo de estado escondido formado pelos resultados das camadas intermédias em fichas anteriores. O seu tamanho é determinado por `mem_len`. Ao utilizar este contexto, esses modelos são capazes de aprender dependências mais longos e também pode ser usado para geração de texto mais rápido em inferência: um modelo transformador normal teria de reexaminar toda a seqüência de índices gerados até agora, enquanto podemos alimentar os novos tokens um por um para um transformador XL (como fazemos com um RNN regular).

In [ ]:
show_doc(TransformerXL.reset)

<h4 id="TransformerXL.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L192" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#TransformerXL-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="TransformerXL-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#TransformerXL-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reset the internal memory.  

## Descodificadores

In [ ]:
show_doc(LinearDecoder, title_level=3)

<h3 id="LinearDecoder" class="doc_header"><code>class</code> <code>LinearDecoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L136" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LinearDecoder-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>LinearDecoder</code>(**`n_out`**:`int`, **`n_hid`**:`int`, **`output_p`**:`float`, **`tie_encoder`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`bias`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="LinearDecoder-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LinearDecoder-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>LinearDecoder</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

To go on top of a RNNCore module and create a Language Model.  

Criar um decodificador para ir em cima de um codificador [`RNNCore`](/text.models.awd_lstm.html#RNNCore) e criar um modelo de linguagem. `N_hid` é a dimensão do último estado escondido do codificador,` n_out` o tamanho da saída. Abandono do `output_p` é aplicado. Se um `tie_encoder` é passado, ele será usado para os pesos da camada linear, que terão` bias` ou não.

In [ ]:
show_doc(PoolingLinearClassifier, title_level=3)

<h3 id="PoolingLinearClassifier" class="doc_header"><code>class</code> <code>PoolingLinearClassifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L230" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PoolingLinearClassifier-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PoolingLinearClassifier</code>(**`layers`**:`Collection`\[`int`\], **`drops`**:`Collection`\[`float`\]) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PoolingLinearClassifier-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PoolingLinearClassifier-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PoolingLinearClassifier</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a linear classifier with pooling.  

A última saída, `MaxPooling` de todas as saídas e` AvgPooling` de todas as saídas são concatenados, então os blocos são empilhados de [`bn_drop_lin`](/layers.html#bn_drop_lin), de acordo com os valores em [`layers`](/layers.html#layers) e `drops`.

## módulos básicos da PNL

No topo da pytorch ou o [`layers`](/layers.html#layers) fastai, os modelos de linguagem usar algumas camadas personalizados específicos para PNL.

In [ ]:
show_doc(EmbeddingDropout, title_level=3)

<h3 id="EmbeddingDropout" class="doc_header"><code>class</code> <code>EmbeddingDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L57" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#EmbeddingDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>EmbeddingDropout</code>(**`emb`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`embed_p`**:`float`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="EmbeddingDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#EmbeddingDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>EmbeddingDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Apply dropout with probabily `embed_p` to an embedding layer `emb`.  

Cada linha da matriz incorporação tem uma probabilidade `embed_p` de ser substituído por zeros, enquanto os outros são rescaled em conformidade.

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[-0.7379, -1.3970, -0.4075, -0.1676,  2.0396,  3.2226,  0.7128],
        [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
        [-3.2579,  2.2972, -1.8704, -0.4090,  2.6477, -1.5015,  0.7158],
        [ 2.1455,  1.0571, -0.6086,  3.5700,  2.6271, -3.1353,  0.7277],
        [-3.7003, -1.8846,  0.2029, -0.6839,  0.2968, -2.0199,  1.3127],
        [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
        [-0.0051,  2.7428,  3.0068,  0.6242,  1.2747,  0.9262,  0.4070],
        [ 1.9312,  3.0524, -1.2806,  1.5910, -2.1789, -0.1636, -3.4924]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, title_level=3)

<h3 id="RNNDropout" class="doc_header"><code>class</code> <code>RNNDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L17" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>RNNDropout</code>(**`p`**:`float`=***`0.5`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="RNNDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>RNNDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Dropout with probability `p` that is consistent on the seq_len dimension.  

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[-2.1156,  0.9734,  0.2428,  0.9396,  0.4072, -0.8197,  0.3718],
          [ 0.4838,  1.3077, -0.8239, -0.6557,  1.3938,  0.6086, -0.2622],
          [ 0.2372, -0.1627,  0.3117, -0.4811, -1.0841, -0.5207, -0.5131]],
 
         [[-0.6924,  0.4122,  0.2517, -1.0120,  0.6808,  0.8800, -0.7463],
          [-0.9498,  0.7655,  0.7471, -0.2767,  1.2155, -0.1042, -2.1443],
          [-1.2342,  1.9187, -0.8481, -0.4115, -1.3223,  1.4266, -1.4150]],
 
         [[ 0.1539,  0.3142,  0.2158,  1.1411,  0.1316,  0.6158, -1.5078],
          [-1.0177, -0.9230,  0.9994,  0.1140,  0.7432,  0.4353,  0.0096],
          [-0.8231,  1.0086,  1.7685,  0.3304, -0.0896, -1.0513, -1.3017]]]),
 tensor([[[-3.0223,  1.3905,  0.0000,  0.0000,  0.5818, -0.0000,  0.5312],
          [ 0.6911,  1.8681, -0.0000, -0.0000,  1.9911,  0.0000, -0.3745],
          [ 0.3389, -0.2324,  0.0000, -0.0000, -1.5487, -0.0000, -0.7331]],
 
         [[-0.9892,  0.5889,  0.3596, -1.4458,  0.9725,  1.2571, -0.0000],
          [-1

In [ ]:
show_doc(WeightDropout, title_level=3)

<h3 id="WeightDropout" class="doc_header"><code>class</code> <code>WeightDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L27" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>WeightDropout</code>(**`module`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`weight_p`**:`float`, **`layer_names`**:`StrList`=***`['weight_hh_l0']`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="WeightDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>WeightDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A module that warps another layer in which some weights will be replaced by 0 during training.  

Aplica-se abandono de probabilidade `weight_p` às camadas em` layer_names` de `module` no modo de treino. Uma cópia desses pesos é mantido para que a máscara de abandono pode mudar em cada lote.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[-0.0702,  0.5725],
        [-0.3910,  0.6512],
        [-0.2203, -0.4315],
        [ 0.2750, -0.2917],
        [-0.4890, -0.3094],
        [ 0.4638, -0.3807],
        [-0.2290, -0.6964],
        [ 0.1224,  0.4043]], requires_grad=True)

É, no início de uma passagem para a frente que o abandono é aplicado aos pesos.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[-0.0000,  0.0000],
        [-0.6517,  0.0000],
        [-0.0000, -0.7191],
        [ 0.4583, -0.0000],
        [-0.0000, -0.0000],
        [ 0.7730, -0.6345],
        [-0.0000, -1.1607],
        [ 0.2040,  0.6739]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(PositionalEncoding, title_level=3)

<h3 id="PositionalEncoding" class="doc_header"><code>class</code> <code>PositionalEncoding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L11" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PositionalEncoding-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PositionalEncoding</code>(**`d`**:`int`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PositionalEncoding-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PositionalEncoding-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PositionalEncoding</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Encode the position with a sinusoid.  

In [ ]:
show_doc(DecoderLayer, title_level=3)

<h3 id="DecoderLayer" class="doc_header"><code>class</code> <code>DecoderLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L138" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DecoderLayer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>DecoderLayer</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="DecoderLayer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DecoderLayer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>DecoderLayer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Basic block of a Transformer model.  

In [ ]:
show_doc(MultiHeadAttention, title_level=3)

<h3 id="MultiHeadAttention" class="doc_header"><code>class</code> <code>MultiHeadAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L33" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiHeadAttention-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MultiHeadAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`=***`None`***, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="MultiHeadAttention-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiHeadAttention-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MultiHeadAttention</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

MutiHeadAttention.  

In [ ]:
show_doc(MultiHeadRelativeAttention, title_level=3)

<h3 id="MultiHeadRelativeAttention" class="doc_header"><code>class</code> <code>MultiHeadRelativeAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiHeadRelativeAttention-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MultiHeadRelativeAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`MultiHeadAttention`](/text.models.transformer.html#MultiHeadAttention)

<div class="collapse" id="MultiHeadRelativeAttention-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiHeadRelativeAttention-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MultiHeadRelativeAttention</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

MutiHeadAttention with relative positional encoding.  

In [ ]:
show_doc(SequentialRNN, title_level=3)

<h3 id="SequentialRNN" class="doc_header"><code>class</code> <code>SequentialRNN</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L153" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialRNN-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>SequentialRNN</code>(**\*`args`**) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

<div class="collapse" id="SequentialRNN-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialRNN-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>SequentialRNN</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A sequential module that passes the reset call to its children.  

In [ ]:
show_doc(SequentialRNN.reset)

<h4 id="SequentialRNN.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L155" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialRNN-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="SequentialRNN-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialRNN-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Chamar o `função reset` de [`self.children`](/torch_core.html#children) (se tiver).

In [ ]:
show_doc(dropout_mask)

<h4 id="dropout_mask" class="doc_header"><code>dropout_mask</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L13" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#dropout_mask-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>dropout_mask</code>(**`x`**:`Tensor`, **`sz`**:`Collection`\[`int`\], **`p`**:`float`)

<div class="collapse" id="dropout_mask-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#dropout_mask-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>dropout_mask</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return a dropout mask of the same type as `x`, size `sz`, with probability `p` to cancel an element.  

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286, 0.0000],
        [1.4286, 1.4286, 1.4286, 0.0000, 1.4286, 0.0000, 0.0000],
        [1.4286, 0.0000, 1.4286, 0.0000, 0.0000, 0.0000, 1.4286]])

Uma tal máscara é, então, expandido na dimensão do comprimento e sequência multiplicado pela entrada fazer uma [`RNNDropout`](/text.models.awd_lstm.html#RNNDropout).

In [ ]:
show_doc(feed_forward)

<h4 id="feed_forward" class="doc_header"><code>feed_forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#feed_forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>feed_forward</code>(**`d_model`**:`int`, **`d_ff`**:`int`, **`ff_p`**:`float`=***`0.0`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***)

<div class="collapse" id="feed_forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#feed_forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>feed_forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

## Indocumentados Métodos - Métodos movidos abaixo desta linha irá intencionalmente ser escondido

In [ ]:
show_doc(WeightDropout.forward)

<h4 id="WeightDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L44" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**\*`args`**:`ArgStar`)

<div class="collapse" id="WeightDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(EmbeddingDropout.forward)

<h4 id="EmbeddingDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L65" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#EmbeddingDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`words`**:`LongTensor`, **`scale`**:`Optional`\[`float`\]=***`None`***) → `Tensor`

<div class="collapse" id="EmbeddingDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#EmbeddingDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(RNNDropout.forward)

<h4 id="RNNDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L22" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**:`Tensor`) → `Tensor`

<div class="collapse" id="RNNDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(WeightDropout.reset)

<h4 id="WeightDropout.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L51" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="WeightDropout-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(PoolingLinearClassifier.forward)

<h4 id="PoolingLinearClassifier.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L240" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PoolingLinearClassifier-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

<div class="collapse" id="PoolingLinearClassifier-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PoolingLinearClassifier-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(LinearDecoder.forward)

<h4 id="LinearDecoder.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L147" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LinearDecoder-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

<div class="collapse" id="LinearDecoder-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LinearDecoder-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## Novos Métodos - Por favor, documento ou mover para a seção em situação irregular